In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

LOADING DATA

In [ ]:
df_train = pd.read_csv("../input/tabular-playground-series-nov-2021/train.csv")
df_test = pd.read_csv("../input/tabular-playground-series-nov-2021/test.csv")


In [ ]:
df_train.head()

In [ ]:
#df_test
#print([i for i in df_test.columns])


In [ ]:
for i in df_train.columns: #to check is theres any missing value present
    print(df_train[i].isnull().sum(),i)

In [ ]:
y = df_train['target']
X=  df_train.drop(columns=["target"])

In [ ]:
X["id"] = pd.to_numeric(X['id'])
X.set_index('id',inplace =True)

In [ ]:
X.info()

In [ ]:
X.isna().any().sum()

In [ ]:
df_test["id"] = pd.to_numeric(df_test['id'])
df_test.set_index('id',inplace =True)


In [ ]:
memory_train = sum(X.memory_usage()) / 1e6
print(f'[INFO] Memory usage train_before: {memory_train:.2f} MB.')

memory_test = sum(df_test.memory_usage()) / 1e6
print(f'[INFO] Memory usage test_before: {memory_test:.2f} MB.\n')

In [ ]:
for i in X.columns:
    if X[i].dtype == "float64":
        X[i] = pd.to_numeric(X[i],downcast="float")
    if X[i].dtype == "int64":
        X[i] = pd.to_numeric(X[i],downcast="integer")

for i in df_test.columns:
    if df_test[i].dtype == "float64":
        df_test[i] = pd.to_numeric(df_test[i],downcast="float")
    if df_test[i].dtype == "int64":
        df_test[i] = pd.to_numeric(df_test[i],downcast="integer")
        
    

In [ ]:
memory_train = sum(X.memory_usage()) / 1e6
print(f'[INFO] Memory usage train: {memory_train:.2f} MB.')

memory_test = sum(df_test.memory_usage()) / 1e6
print(f'[INFO] Memory usage test: {memory_test:.2f} MB.')

In [ ]:
import  matplotlib.pyplot as plt
import seaborn as sns

****EXPLORING PDFs OF ALL FEATURES****

Converted some of the distributions into normal one, but shockingly, after doing feature engineering , the AUC was limited to .70 . But as these features were passed just after scaling into ANN classifier, results were much better( 0.7493 auc) .Hence feature engineering is of no use on this synthetic dataset

In [ ]:
fig, axes = plt.subplots(10,10, figsize=(20, 12))
axes = axes.flatten()

for idx, ax in enumerate(axes):
    sns.kdeplot(
        data=X, ax=ax, hue= y , fill=True, x=f'f{idx}', palette=['#4DB6AC', 'red'], legend=idx==0)
 
plt.show()

In [ ]:
import tensorflow as tf
import keras
from tensorflow.keras.layers import Dense,Activation,BatchNormalization,Dropout,Input,Multiply,Concatenate,Add
from tensorflow.keras.models import Sequential,Model
tf.random.set_seed(100)

****ROBUST SCALER GAVE BETTER RESULTS THAN STANDARD SCALER AS IT PENALIZES OUTLIERS****

In [ ]:

from sklearn.preprocessing import RobustScaler
rb = RobustScaler()


In [ ]:
X_scaled= rb.fit_transform(X)


In [ ]:
X_scaled_cleaned = pd.DataFrame(X_scaled, columns= ['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19', 'f20', 'f21', 'f22', 'f23', 'f24', 'f25', 'f26', 'f27', 'f28', 'f29', 'f30', 'f31', 'f32', 'f33', 'f34', 'f35', 'f36', 'f37', 'f38', 'f39', 'f40', 'f41', 'f42', 'f43', 'f44', 'f45', 'f46', 'f47', 'f48', 'f49', 'f50', 'f51', 'f52', 'f53', 'f54', 'f55', 'f56', 'f57', 'f58', 'f59', 'f60', 'f61', 'f62', 'f63', 'f64', 'f65', 'f66', 'f67', 'f68', 'f69', 'f70', 'f71', 'f72', 'f73', 'f74', 'f75', 'f76', 'f77', 'f78', 'f79', 'f80', 'f81', 'f82', 'f83', 'f84', 'f85', 'f86', 'f87', 'f88', 'f89', 'f90', 'f91', 'f92', 'f93', 'f94', 'f95', 'f96', 'f97', 'f98', 'f99'])

**CUSTOM ANN FOR BETTER AUC SCORE**

A custom ANN was made using skip connections. Skip connections preserves the knowledge and avoid vanishing gradient problems in deep network. Also multiplication of features was considered instead of addition as addition badly hampers the correlation between features. Batch normalisation is used along with drop-out layers to prevent overfitting.

In [ ]:
# ROBUST, TRANSFORM, BEST MODEL

def create_model():
    x_input = Input(shape=(X_scaled_cleaned.shape[1],))
    
    x1 = Dense(100,activation='relu' )(x_input)
    x2 = Dense(128, activation='relu')(x1)
    x3 = Dense(256,activation='relu' )(x2)
    z2 = Dense(100,activation='relu')(x3)
    
    z31 = Multiply()([x1, z2])
    z31 = BatchNormalization()(z31)
    
    z31 = Dropout(0.3)(z31)
    
    x4 =  Dense(128,activation='relu')(z31)
    x5 =  Dense(64,activation='relu')(x4)
    z3=  Dense(128,activation='relu')(x5)
    
    z32 = Multiply()([x4, z3])
    z32 = BatchNormalization()(z32)
  
    z32 = Dropout(0.3)(z32)

    x7 =  Dense(32,activation='relu')(z32)
    
#     x8 =  Dense(32,activation='relu')(x7)
#     x9 =  Dense(16,activation='relu')(x8)
#     z4 =  Dense(100,activation='relu')(x9)
#     z33 = Concatenate()([x1,x4, z4])
#     z33 = BatchNormalization()(z33)
#     x10 =  Dense(8,activation='relu')(x9)
    
#     z41 = Multiply()([x3, z3])
#     z41 = BatchNormalization()(z41)
    
#     z4 =  Dense(128,activation='relu')(z41)
#     z51 = Multiply()([x5, z4])
#     z51 = BatchNormalization()(z51)
    
#     z4 =  Dense(64,activation='relu')(z41)
#     x =  Concatenate()([z2, z3, z4])
#     x =  Dense(32, activation='relu')(x)
    x_output = Dense(units=1,activation='sigmoid')(x7)
    model = Model(inputs=x_input, outputs=x_output,name='Model')
    return model


In [ ]:
# pca = PCA(n_components=0.98)
# X_reduced = pca.fit_transform(X_scaled_cleaned)


In [ ]:
# X_reduced.shape

In [ ]:
model = create_model()
model.summary()


In [ ]:
from tensorflow.keras.utils import plot_model

plot_model(model,  to_file='TPS_Model.png', show_shapes=True,show_layer_names=True)


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping,ReduceLROnPlateau
early = EarlyStopping(monitor="loss", mode= "min", patience=3)
learning_rate_reduction = ReduceLROnPlateau( monitor="auc",patience =3, verbose=1 , factor=0.2, min_learning_rate=0.0001 )
callbacks_list = [early,learning_rate_reduction]
# Compile model
model.compile(optimizer='adam',loss='binary_crossentropy', metrics=['AUC']) 
# history = model.fit(train_features,train_labels_encoded, batch_size =100 , callbacks= callbacks_list , class_weight = cw , epochs=100, validation_data= (test_features,test_labels_encoded))
history = model.fit(X_scaled_cleaned, y , callbacks= callbacks_list,batch_size = 100 , epochs=100)

In [ ]:
test = rb.transform(df_test)

In [ ]:
test


In [ ]:
X_test_cleaned = pd.DataFrame(test, columns= ['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19', 'f20', 'f21', 'f22', 'f23', 'f24', 'f25', 'f26', 'f27', 'f28', 'f29', 'f30', 'f31', 'f32', 'f33', 'f34', 'f35', 'f36', 'f37', 'f38', 'f39', 'f40', 'f41', 'f42', 'f43', 'f44', 'f45', 'f46', 'f47', 'f48', 'f49', 'f50', 'f51', 'f52', 'f53', 'f54', 'f55', 'f56', 'f57', 'f58', 'f59', 'f60', 'f61', 'f62', 'f63', 'f64', 'f65', 'f66', 'f67', 'f68', 'f69', 'f70', 'f71', 'f72', 'f73', 'f74', 'f75', 'f76', 'f77', 'f78', 'f79', 'f80', 'f81', 'f82', 'f83', 'f84', 'f85', 'f86', 'f87', 'f88', 'f89', 'f90', 'f91', 'f92', 'f93', 'f94', 'f95', 'f96', 'f97', 'f98', 'f99'])

In [ ]:
target = model.predict(X_test_cleaned)

In [ ]:
ids = pd.read_csv("../input/tabular-playground-series-nov-2021/sample_submission.csv")
ids = ids['id']
data = {
  "id": ids,
  "target": target[:,-1]
}

submission = pd.DataFrame(data)

In [ ]:
submission

In [ ]:
submission.to_csv("submission2.csv",index =False)

OUTLIER DETECTIONS IN FEATURES 

In [ ]:

# fig, axes = plt.subplots(100,1,figsize=(10,300))
# for i,j in zip(X_scaled.columns,range(0,100)):
#     axes[j].set_title(i)
#     axes[j].boxplot(X_scaled[i])


PDF DISTRIBUTIONS OF ALL FEATURES

In [ ]:
# fig, axes = plt.subplots(100,1,figsize=(10,300))
# for i,j in zip(X_scaled.columns,range(0,100)):
#     axes[j].set_title(i)
#     axes[j].hist(X_scaled[i],bins = 70)


In [ ]:
# first_quantile = list(X_scaled.quantile(0.10))
# second_quantile = list(X_scaled.quantile(0.90))
# # print(first_quantile)
# # print(second_quantile)

In [ ]:

#  # IQR
# Q1 = X_scaled.quantile(0.25)
# Q3 = X_scaled.quantile(0.75)
# IQR = Q3 - Q1
# #  print("Old Shape: ", X_scaled.shape)
# lower = Q1 - 1.5*IQR
# upper = Q3 + 1.5*IQR
# # X_scaled.where(upper[0], inplace = True)
# # X_scaled.drop(lower[0], inplace = True)
# # print("New Shape: ", X_scaled.shape)
# X_clean = np.where((X_scaled >lower)&(X_scaled <upper),X_scaled, np.mean(X_scaled))


In [ ]:
# X_scaled_cleaned = pd.DataFrame(X_scaled, columns= ['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19', 'f20', 'f21', 'f22', 'f23', 'f24', 'f25', 'f26', 'f27', 'f28', 'f29', 'f30', 'f31', 'f32', 'f33', 'f34', 'f35', 'f36', 'f37', 'f38', 'f39', 'f40', 'f41', 'f42', 'f43', 'f44', 'f45', 'f46', 'f47', 'f48', 'f49', 'f50', 'f51', 'f52', 'f53', 'f54', 'f55', 'f56', 'f57', 'f58', 'f59', 'f60', 'f61', 'f62', 'f63', 'f64', 'f65', 'f66', 'f67', 'f68', 'f69', 'f70', 'f71', 'f72', 'f73', 'f74', 'f75', 'f76', 'f77', 'f78', 'f79', 'f80', 'f81', 'f82', 'f83', 'f84', 'f85', 'f86', 'f87', 'f88', 'f89', 'f90', 'f91', 'f92', 'f93', 'f94', 'f95', 'f96', 'f97', 'f98', 'f99'])

In [ ]:
# X_scaled_cleaned.isnull().sum()

In [ ]:
# for i,j in zip(X_scaled.columns,second_quantile):
#     X_scaled[i] = np.where(X_scaled[i]>j,np.median(X_scaled[i]),X_scaled[i])
# a = X_clean['f0'].fillna(np.median(X_clean['f0']))

In [ ]:

# = [ "f0","f2","f4","f9","f12","f16","f19","f20","f21","f23","f24","f27","f28","f30","f31","f32","f33","f35","f36","f39","f42","f44","f46","f48","f49","f51","f52","f53","f58","f59","f60","f61","f62","f63","f64","f68","f69","f72","f73","f76","f78","f79","f81","f83","f84","f87","f88","f89","f90","f92","f93","f94","f95","f98","f99"]

In [ ]:
# for i in l:
#     print("mean:", np.mean(X_scaled[i]))
#     print("median:", np.median(X_scaled[i]))

In [ ]:
# first =[]
# second =[]
# for i in l:
#     first_quantile = X_scaled[i].quantile(0.10)
#     first.append(first_quantile)
#     second_quantile = X_scaled[i].quantile(0.90)
#     second.append(second_quantile)
# print(first)

In [ ]:
# first_quantile = X_scaled.quantile(0.10)
# second_quantile = X_scaled.quantile(0.90)


In [ ]:
# for i,j in zip(X_scaled.columns, first_quantile):
#     X_scaled[i] = np.where(X_scaled[i]<j,np.median(X_scaled[i]),X_scaled[i])


In [ ]:
# for i,j in zip(X_scaled.columns ,second_quantile):
#     X_scaled[i] = np.where(X_scaled[i]>j,np.median(X_scaled[i]),X_scaled[i])


In [ ]:
# from sklearn.ensemble import IsolationForest

# iso = IsolationForest(contamination=0.1)
# yhat = iso.fit_predict(X_scaled)

In [ ]:
# fig, axes = plt.subplots(100,1,figsize=(10,300))
# for i,j in zip(X_clean.columns,range(0,100)):
#     axes[j].set_title(i)
#     axes[j].boxplot(X_clean[i])


PDFs POST HANDLING THE OUTLIERS

In [ ]:
# fig, axes = plt.subplots(100,1,figsize=(10,300))
# for i,j in zip(X_scaled_cleaned.columns,range(0,100)):
#     axes[j].set_title(i)
#     axes[j].hist(X_scaled_cleaned[i],bins = 200)

In [ ]:
# to find out features that are negatively  correlated with target so that we can remove them

# x = [j for i,j in zip(X.corr()["target"],X.corr()["target"].index) if i <0]
# print(x)

In [ ]:
# #FUNCTION FOR REMOVING OUTLIERS AND IQR
# outliers = []
# def detect_outliers_zscore(data):
#     thres = 3
#     mean = np.mean(data)
#     std = np.std(data)
#     # print(mean, std)
#     for i in data:
#         z_score = (i-mean)/std
#         if (np.abs(z_score) > thres):
#             outliers.append(i)
#     return outliers# Driver code





In [ ]:

# Computing 10th, 90th percentiles and replacing the outliers


# mean = np.mean(X_scaled['f4'])# Replace with median
# for i in sample_outliers:
#     c = np.where(X_scaled['f4'] ==i ,mean , X_scaled['f4'])
# print("New array: ",c)

In [ ]:

# X_scaled_cleaned = X_scaled.drop(['f1', 'f2', 'f3', 'f5', 'f9', 'f14', 'f15', 'f16', 'f17', 'f21', 'f22', 'f23', 'f25', 'f26', 'f31', 'f36', 'f37', 'f38', 'f39', 'f47', 'f48', 'f49', 'f53', 'f54', 'f55', 'f59', 'f62', 'f63', 'f64', 'f65', 'f66', 'f67', 'f71', 'f72', 'f75', 'f76', 'f78', 'f80', 'f82', 'f83', 'f84', 'f86', 'f87', 'f90', 'f91', 'f92', 'f93', 'f94', 'f97', 'f99'],axis =1)

In [ ]:
# from sklearn.ensemble import ExtraTreesClassifier
# ex = ExtraTreesClassifier()
# ex.fit(X_scaled_cleaned,y)
# print(ex.feature_importances_)

In [ ]:
# feat_importances = pd.Series(ex.feature_importances_,index =X_scaled_cleaned.columns)
# feat_importances.nlargest(50).plot(kind='barh')

In [ ]:
# top_features = ['f34',
#  'f43',
#  'f8',
#  'f50',
#  'f27',
#  'f41',
#  'f57',
#  'f96',
#  'f40',
#  'f81',
#  'f10',
#  'f45',
#  'f77',
#  'f70',
#  'f11',
#  'f6',
#  'f74',
#  'f85',
#  'f7',
#  'f18']
# list(set(X_scaled_cleaned.columns).difference(top_features))

In [ ]:
# X_scaled_cleaned = X_scaled_cleaned.drop(['f19',
#  'f56',
#  'f89',
#  'f73',
#  'f0',
#  'f88',
#  'f42',
#  'f29',
#  'f4',
#  'f52',
#  'f60',
#  'f33',
#  'f28',
#  'f32',
#  'f69',
#  'f95',
#  'f35',
#  'f24',
#  'f30',
#  'f12',
#  'f51',
#  'f44',
#  'f46',
#  'f98',
#  'f13',
#  'f79',
#  'f20',
#  'f58',
#  'f68',
#  'f61'],axis =1)

# X_scaled1 = X_scaled.drop(l,axis=1)
# print(X_scaled1.shape)

In [ ]:
# from sklearn.utils.class_weight import compute_class_weight
# weights = compute_class_weight('balanced', np.unique(np.array(y)),np.array(y))
# cw = dict(zip( np.unique(y), weights))
# print(cw)

In [ ]:
# from sklearn.preprocessing import StandardScaler
# rb = StandardScaler()
# X_scaled= rb.fit_transform(X)

In [ ]:
# X_scaled_cleaned = pd.DataFrame(X_scaled, columns= ['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19', 'f20', 'f21', 'f22', 'f23', 'f24', 'f25', 'f26', 'f27', 'f28', 'f29', 'f30', 'f31', 'f32', 'f33', 'f34', 'f35', 'f36', 'f37', 'f38', 'f39', 'f40', 'f41', 'f42', 'f43', 'f44', 'f45', 'f46', 'f47', 'f48', 'f49', 'f50', 'f51', 'f52', 'f53', 'f54', 'f55', 'f56', 'f57', 'f58', 'f59', 'f60', 'f61', 'f62', 'f63', 'f64', 'f65', 'f66', 'f67', 'f68', 'f69', 'f70', 'f71', 'f72', 'f73', 'f74', 'f75', 'f76', 'f77', 'f78', 'f79', 'f80', 'f81', 'f82', 'f83', 'f84', 'f85', 'f86', 'f87', 'f88', 'f89', 'f90', 'f91', 'f92', 'f93', 'f94', 'f95', 'f96', 'f97', 'f98', 'f99'])

In [ ]:
# model = create_model()
# model.summary()


In [ ]:


# model = Sequential()
# model.add(Dense(100,input_shape = (X_scaled_cleaned.shape[1],)))

# model.add(Dense(100, activation='relu'))
# model.add(keras.layers.BatchNormalization())
# model.add(Dropout(0.3))

# model.add(Dense(128, activation='relu'))
# model.add(keras.layers.BatchNormalization())
# model.add(Dropout(0.3))

# # model.add(Dense(256, activation='relu'))
# # model.add(keras.layers.BatchNormalization())
# # model.add(Dropout(0.3))

# # model.add(Dense(512, activation='relu'))
# # model.add(keras.layers.BatchNormalization())
# # model.add(Dropout(0.3))

# # model.add(Dense(256, activation='relu'))
# # model.add(keras.layers.BatchNormalization())
# # model.add(Dropout(0.3))



# # model.add(Dense(128, activation='relu'))
# # model.add(keras.layers.BatchNormalization())
# # model.add(Dropout(0.3))


# model.add(Dense(64, activation='relu'))
# model.add(keras.layers.BatchNormalization())
# model.add(Dropout(0.3))


# model.add(Dense(32, activation='relu'))
# model.add(keras.layers.BatchNormalization())
# model.add(Dropout(0.3))


# model.add(Dense(16, activation='relu'))
# model.add(keras.layers.BatchNormalization())
# model.add(Dropout(0.3))

# model.add(Dense(8, activation='relu'))
# model.add(keras.layers.BatchNormalization())
# model.add(Dropout(0.3))

# model.add(Dense(1,activation="sigmoid"))
# model.summary()

In [ ]:
# !pip install scikit-learn  -U

In [ ]:
# from sklearn.ensemble import RandomForestClassifier
# rf = RandomForestClassifier()

In [ ]:
#HyperParameters

# n_estimators = [int(i) for i in np.linspace(start = 100 ,stop= 1200, num =12)]
# max_features = ['auto','sqrt']
# max_depth = [int(i) for i in np.linspace(5,30,num =6) ]
# min_samples_split = [2,3,10,15,100]
# min_samples_leaf =[1,2,5,10]

In [ ]:
# from sklearn.model_selection import RandomizedSearchCV
# random_grid = {'n_estimators': n_estimators,
#               'max_features': max_features,
#               'max_depth': max_depth,
#               'min_samples_split': min_samples_split,
#               'min_samples_leaf': min_samples_leaf}
# print(random_grid)

In [ ]:
#rf_random = RandomizedSearchCV(estimator=rf,param_distributions=random_grid,scoring='neg_mean_squared_error',n_iter =5,random_state=42,n_jobs =1,verbose =2)

In [ ]:
# # rf_random.fit(X_scaled_cleaned,y)
# df_test = pd.read_csv("../input/tabular-playground-series-nov-2021/test.csv")


In [ ]:
# df_test["id"] = pd.to_numeric(df_test['id'])
# df_test.set_index('id',inplace =True)


In [ ]:

# from sklearn.preprocessing import RobustScaler
# test_scaled= RobustScaler().fit_transform(df_test)

In [ ]:
#X_test = pd.DataFrame(test_scaled, columns= ['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19', 'f20', 'f21', 'f22', 'f23', 'f24', 'f25', 'f26', 'f27', 'f28', 'f29', 'f30', 'f31', 'f32', 'f33', 'f34', 'f35', 'f36', 'f37', 'f38', 'f39', 'f40', 'f41', 'f42', 'f43', 'f44', 'f45', 'f46', 'f47', 'f48', 'f49', 'f50', 'f51', 'f52', 'f53', 'f54', 'f55', 'f56', 'f57', 'f58', 'f59', 'f60', 'f61', 'f62', 'f63', 'f64', 'f65', 'f66', 'f67', 'f68', 'f69', 'f70', 'f71', 'f72', 'f73', 'f74', 'f75', 'f76', 'f77', 'f78', 'f79', 'f80', 'f81', 'f82', 'f83', 'f84', 'f85', 'f86', 'f87', 'f88', 'f89', 'f90', 'f91', 'f92', 'f93', 'f94', 'f95', 'f96', 'f97', 'f98', 'f99'])

In [ ]:
# X_test_cleaned = X_test.drop(['f1', 'f2', 'f3', 'f5', 'f9', 'f14', 'f15', 'f16', 'f17', 'f21', 'f22', 'f23', 'f25', 'f26', 'f31', 'f36', 'f37', 'f38', 'f39', 'f47', 'f48', 'f49', 'f53', 'f54', 'f55', 'f59', 'f62', 'f63', 'f64', 'f65', 'f66', 'f67', 'f71', 'f72', 'f75', 'f76', 'f78', 'f80', 'f82', 'f83', 'f84', 'f86', 'f87', 'f90', 'f91', 'f92', 'f93', 'f94', 'f97', 'f99'],axis =1)

In [ ]:
# target = model.predict(X_test_cleaned)

In [ ]:
# #  # IQR
# Q1 = X_test.quantile(0.25)
# Q3 = X_test.quantile(0.75)
# IQR = Q3 - Q1
# #  print("Old Shape: ", X_scaled.shape)
# lower = Q1 - 1.5*IQR
# upper = Q3 + 1.5*IQR
# # X_scaled.where(upper[0], inplace = True)
# # X_scaled.drop(lower[0], inplace = True)
# # print("New Shape: ", X_scaled.shape)
# X_clean_test = np.where((X_test >lower)&(X_test <upper),X_test, np.mean(X_test))


In [ ]:
# X_t = pd.DataFrame(X_clean_test, columns= ['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19', 'f20', 'f21', 'f22', 'f23', 'f24', 'f25', 'f26', 'f27', 'f28', 'f29', 'f30', 'f31', 'f32', 'f33', 'f34', 'f35', 'f36', 'f37', 'f38', 'f39', 'f40', 'f41', 'f42', 'f43', 'f44', 'f45', 'f46', 'f47', 'f48', 'f49', 'f50', 'f51', 'f52', 'f53', 'f54', 'f55', 'f56', 'f57', 'f58', 'f59', 'f60', 'f61', 'f62', 'f63', 'f64', 'f65', 'f66', 'f67', 'f68', 'f69', 'f70', 'f71', 'f72', 'f73', 'f74', 'f75', 'f76', 'f77', 'f78', 'f79', 'f80', 'f81', 'f82', 'f83', 'f84', 'f85', 'f86', 'f87', 'f88', 'f89', 'f90', 'f91', 'f92', 'f93', 'f94', 'f95', 'f96', 'f97', 'f98', 'f99'])